In [4]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Load the daily prices and portfolio holdings data
prices_df = pd.read_csv('DailyPrices.csv')
holdings_df = pd.read_csv('Portfolio.csv')
prices_df.set_index('Date', inplace=True)
prices_df = prices_df.pct_change().dropna()

In [276]:
# Create an empty dictionary to store the portfolio dataframes
portfolios = {}

# Loop over each portfolio
for portfolio in holdings_df['Portfolio'].unique():
    # Subset the holdings data for the current portfolio
    stocks = holdings_df[holdings_df['Portfolio'] == portfolio]['Stock']
    holdings_subset = holdings_df[holdings_df['Portfolio'] == portfolio]
    holdings_subset.set_index('Stock',inplace = True)
    holdings_subset = holdings_subset.T
    portfolio_ = pd.DataFrame()
    # Calculate the portfolio value for each day and store it
    for i in holdings_subset.columns:
        if i in prices_df.columns:
            portfolio_[i] = prices_df[i] * holdings_df[holdings_df['Stock'] == i]['Holding'].values 
    portfolio_returns = portfolio_.sum(axis=1)
    portfolios[portfolio] = portfolio_returns

In [277]:
# Combine the portfolio dataframes into a single dataframe
total_df = pd.concat(portfolios.values(), axis=1, keys=portfolios.keys())
# Calculate the portfolio returns
returns = total_df.pct_change().dropna()
# Calculate the exponentially weighted covariance matrix
lambda_094 = 0.94
cov_ewm = returns.ewm(alpha=lambda_094, adjust=False).cov()
cov_ewm = cov_ewm.dropna()
cov_ewm

alpha = 0.01
cov_ewm = cov_ewm.reset_index()
A = np.sqrt(np.dot(portfolios['A'][2:], np.dot(cov_ewm[cov_ewm['level_1'] == 'A']['A'], portfolios['A'][2:])))*1.645
B = np.percentile(A, 100 * alpha)

In [5]:
# Create an empty dictionary to store the portfolio dataframes
VaR = {}
total_var = 0
# Loop over each portfolio
for portfolio in holdings_df['Portfolio'].unique():
    # Subset the holdings data for the current portfolio
    stocks = holdings_df[holdings_df['Portfolio'] == portfolio]['Stock']
    holdings_subset = holdings_df[holdings_df['Portfolio'] == portfolio]
    holdings_subset.set_index('Stock',inplace = True)
    holdings_subset = holdings_subset.T
    portfolio_ = pd.DataFrame()
    # Calculate the portfolio value for each day and store it
    for i in holdings_subset.columns:
        if i in prices_df.columns:
            portfolio_[i] = prices_df[i] * holdings_df[holdings_df['Stock'] == i]['Holding'].values
    
    total_value = portfolio_.sum(axis=1)
    weights_df = portfolio_.div(total_value, axis=0)
    cov_ewm = portfolio_.ewm(alpha=0.94, adjust=False).cov().dropna()
    # Calculate the portfolio variance as the weighted average of the diagonal elements of the covariance matrix
    portfolio_var = ((cov_ewm.mul(weights_df, axis=0).sum(axis=1) * weights_df.sum(axis=1)).sum())

    # Calculate the portfolio volatility as the square root of the portfolio variance
    portfolio_vol = np.sqrt(portfolio_var)/ np.sqrt(252)

    # Calculate the portfolio VaR
    portfolio_var = (-(weights_df.T.dot(total_value) * portfolio_vol * norm.ppf(0.01))).sum()
    VaR[portfolio] = portfolio_var
    print(f"Portfolio {portfolio} VaR: ${portfolio_var:,.2f}")

for i in VaR.values():
    total_var = total_var + i
print(f"Total VaR: ${total_var:,.2f}")


Portfolio A VaR: $1,519.52
Portfolio B VaR: $2,292.02
Portfolio C VaR: $-2,660.62
Total VaR: $1,150.91


In [647]:
total_var = 0
for i in VaR.values():
    total_var = total_var + i

In [648]:
total_var

1150.9148478392399